# AAI614: Data Science & its Applications

*Notebook 3.2: Practice with Data Cleaning*

<a href="https://colab.research.google.com/github/harmanani/AAI614/blob/main/Week%203/Notebook3.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Exercise I. Load the following datafile from GitHub

In [2]:
grads = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

In [3]:
grads

,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,NaN,NYU,3.0,$40k
1,Jerry,35,3.5,Columbia,5.0,$80k
2,Elaine,55,4.0,Columbia,4.2,$60k
3,Cosmo,5,2.0,City College,2.0,$25k
4,Newman,25,2.8,City College,0.0,$50k
5,Frank,35,3.0,Festivus Uni,NaN,$40k
6,Estelle,100,3.2,Festivus Uni,1.7,$0k
7,Leo,15,2.4,Festivus Uni,0.0,$35k
8,Rachel,50,4.0,Columbia,NaN,$75k


Question 1: Identify all the outliers in the above data.  Justify your answers using objective measures.

In [5]:
# Convert numeric columns to appropriate types
grads["Salary"] = grads["Salary"].str.replace("$", "").str.replace("k", "").astype(float) * 1000
grads["GPA"] = pd.to_numeric(grads["GPA"], errors="coerce")
grads["Sense of Humour (0-5)"] = pd.to_numeric(grads["Sense of Humour (0-5)"], errors="coerce")

In [6]:
# checkign for outliers using IQR for numerical columns
numerical_cols = ["Avg Hours Studies per Week", "GPA", "Sense of Humour (0-5)", "Salary"]
outlier_details = {}

for col in numerical_cols:
    Q1 = grads[col].quantile(0.25)
    Q3 = grads[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # identify outliers
    outliers = grads[(grads[col] < lower_bound) | (grads[col] > upper_bound)][col]
    # store details
    outlier_details[col] = {
        "Q1": Q1,
        "Q3": Q3,
        "IQR": IQR,
        "Lower Bound": lower_bound,
        "Upper Bound": upper_bound,
        "Outliers": outliers.tolist(),
    }

# display results
for col, details in outlier_details.items():
    print(f"Column: {col}")
    print(f"  Q1: {details['Q1']}")
    print(f"  Q3: {details['Q3']}")
    print(f"  IQR: {details['IQR']}")
    print(f"  Lower Bound: {details['Lower Bound']}")
    print(f"  Upper Bound: {details['Upper Bound']}")
    print(f"  Outliers: {details['Outliers']}")
    print("-" * 40)

Column: Avg Hours Studies per Week
  Q1: 20.0
  Q3: 50.0
  IQR: 30.0
  Lower Bound: -25.0
  Upper Bound: 95.0
  Outliers: [100]
----------------------------------------
Column: GPA
  Q1: 2.6999999999999997
  Q3: 3.625
  IQR: 0.9250000000000003
  Lower Bound: 1.3124999999999993
  Upper Bound: 5.0125
  Outliers: []
----------------------------------------
Column: Sense of Humour (0-5)
  Q1: 0.85
  Q3: 3.6
  IQR: 2.75
  Lower Bound: -3.275
  Upper Bound: 7.725
  Outliers: []
----------------------------------------
Column: Salary
  Q1: 35000.0
  Q3: 60000.0
  IQR: 25000.0
  Lower Bound: -2500.0
  Upper Bound: 97500.0
  Outliers: []
----------------------------------------


so there's only one outlier where the average hours studied per week was 100h.

Question 2: There are various data that are missing.  Fill-in the missing data or delete the rows/columns that you think you should delete.  Justify your answer

In [7]:
# fill missing GPA with mean
grads["GPA"].fillna(grads["GPA"].mean(), inplace=True)

# drop rows where 'Sense of Humor' is missing
grads.dropna(subset=["Sense of Humour (0-5)"], inplace=True)

# reset index after dropping rows
grads = grads.reset_index(drop=True)

grads

C:\Users\bay\AppData\Local\Temp\ipykernel_15588\3422541848.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  grads["GPA"].fillna(grads["GPA"].mean(), inplace=True)


,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,3.1125,NYU,3.0,40000.0
1,Jerry,35,3.5000,Columbia,5.0,80000.0
2,Elaine,55,4.0000,Columbia,4.2,60000.0
3,Cosmo,5,2.0000,City College,2.0,25000.0
4,Newman,25,2.8000,City College,0.0,50000.0
5,Estelle,100,3.2000,Festivus Uni,1.7,0.0
6,Leo,15,2.4000,Festivus Uni,0.0,35000.0


1. I converted the salary, GPA, and Sense of Humour to numeric amount before for analysis
2. I replaced missing values in GPA with the meas of the column
3. dropped rows with missing sense of humour since this feature is not critical to the analyssi

Question 3: Reload the data and fill-in the data using mean method as well as the frequent method.

In [8]:
data = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

# Convert numeric columns to appropriate types
data["Salary"] = data["Salary"].str.replace("$", "").str.replace("k", "").astype(float) * 1000
data["GPA"] = pd.to_numeric(data["GPA"], errors="coerce")
data["Sense of Humour (0-5)"] = pd.to_numeric(data["Sense of Humour (0-5)"], errors="coerce")

# Fill missing data using the mean method
data_mean_filled = data.copy()
data_mean_filled["GPA"].fillna(data["GPA"].mean(), inplace=True)
data_mean_filled["Sense of Humour (0-5)"].fillna(data["Sense of Humour (0-5)"].mean(), inplace=True)

# Fill missing data using the frequent method (mode)
data_frequent_filled = data.copy()
data_frequent_filled["GPA"].fillna(data["GPA"].mode()[0], inplace=True)
data_frequent_filled["Sense of Humour (0-5)"].fillna(data["Sense of Humour (0-5)"].mode()[0], inplace=True)

data_frequent_filled

C:\Users\bay\AppData\Local\Temp\ipykernel_15588\1323354674.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_mean_filled["GPA"].fillna(data["GPA"].mean(), inplace=True)
C:\Users\bay\AppData\Local\Temp\ipykernel_15588\1323354674.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,4.0,NYU,3.0,40000.0
1,Jerry,35,3.5,Columbia,5.0,80000.0
2,Elaine,55,4.0,Columbia,4.2,60000.0
3,Cosmo,5,2.0,City College,2.0,25000.0
4,Newman,25,2.8,City College,0.0,50000.0
5,Frank,35,3.0,Festivus Uni,0.0,40000.0
6,Estelle,100,3.2,Festivus Uni,1.7,0.0
7,Leo,15,2.4,Festivus Uni,0.0,35000.0
8,Rachel,50,4.0,Columbia,0.0,75000.0


Exercise II. Run the cell below to create a new dataframe called `df_miss`.  Its first column will contain some missing values.

In [9]:
import pandas as pd
import numpy as np
import random

nrows = 10
ncols = 5

# set a seed for random number generation
np.random.seed(314)
# create an array filled with random data
data = np.array(np.random.rand(nrows, ncols))
# put the data to a pandas dataframe
df_miss = pd.DataFrame(data)
# rename the columns
df_miss.columns = ['col_'+str(ii) for ii in range(ncols)]

# randomly set some values to missing
ix0 = np.random.randint(nrows, size=3)
ix1 = np.random.randint(nrows, size=3)

df_miss['col_0'][ix0] = np.nan
df_miss['col_1'][ix1] = np.nan

print(df_miss)

      col_0     col_1     col_2     col_3     col_4
0       NaN       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3       NaN       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5       NaN  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


C:\Users\bay\AppData\Local\Temp\ipykernel_15588\310065607.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_miss['col_0'][ix0] = np.nan
C:\Users\bay\AppData\Local\Temp\ipykernel_15588\310065607.py:22: FutureWarning: ChainedAssignmentEr

Impute the missing values (NaN) in `col_0` (but not `col_1`) with the median.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [10]:
df_miss['col_0'].fillna(df_miss['col_0'].median(), inplace=True)

df_miss

C:\Users\bay\AppData\Local\Temp\ipykernel_15588\1332160194.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_0'].fillna(df_miss['col_0'].median(), inplace=True)


,col_0,col_1,col_2,col_3,col_4
0,0.677205,NaN,0.265048,0.783205,0.918001
1,0.827355,NaN,0.260480,0.911763,0.260757
2,0.766376,0.261531,0.122291,0.386006,0.840081
3,0.677205,NaN,0.633110,0.584766,0.581232
4,0.677205,0.687155,0.438927,0.320927,0.570552
5,0.677205,0.861074,0.834805,0.105766,0.060408
6,0.596882,0.792395,0.226356,0.535201,0.136066
7,0.372244,0.151977,0.429822,0.792706,0.406957
8,0.177850,0.909252,0.545331,0.100497,0.718721
9,0.978429,0.309776,0.260126,0.662900,0.139720


Impute the missing values in `col_1` with value 0.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [11]:
df_miss['col_1'].fillna(0, inplace=True)

df_miss

C:\Users\bay\AppData\Local\Temp\ipykernel_15588\2032120352.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_1'].fillna(0, inplace=True)


,col_0,col_1,col_2,col_3,col_4
0,0.677205,0.000000,0.265048,0.783205,0.918001
1,0.827355,0.000000,0.260480,0.911763,0.260757
2,0.766376,0.261531,0.122291,0.386006,0.840081
3,0.677205,0.000000,0.633110,0.584766,0.581232
4,0.677205,0.687155,0.438927,0.320927,0.570552
5,0.677205,0.861074,0.834805,0.105766,0.060408
6,0.596882,0.792395,0.226356,0.535201,0.136066
7,0.372244,0.151977,0.429822,0.792706,0.406957
8,0.177850,0.909252,0.545331,0.100497,0.718721
9,0.978429,0.309776,0.260126,0.662900,0.139720
